In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy as sp

In [2]:
df = pd.read_csv('retail_price.csv')

In [3]:
df['month'] = pd.to_datetime(df['month_year'], dayfirst=True).dt.month
df['year'] = pd.to_datetime(df['month_year'], dayfirst = True).dt.year

In [4]:
df[df['product_category_name'] == 'bed_bath_table'][df['month'] == 5][df['year'] == 2017]

/tmp/ipykernel_40450/2229989431.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['product_category_name'] == 'bed_bath_table'][df['month'] == 5][df['year'] == 2017]


,product_id,product_category_name,month_year,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,...,comp_1,ps1,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,lag_price
0,bed1,bed_bath_table,01-05-2017,1,45.95,15.100000,45.95,39,161,2,...,89.9,3.9,15.011897,215.0,4.4,8.76,45.95,4.0,15.1,45.90
441,bed2,bed_bath_table,01-05-2017,58,5214.20,15.011897,89.90,54,245,1,...,89.9,3.9,15.011897,215.0,4.4,8.76,45.95,4.0,15.1,89.90
671,bed5,bed_bath_table,01-05-2017,1,215.00,8.760000,215.00,56,162,5,...,89.9,3.9,15.011897,215.0,4.4,8.76,45.95,4.0,15.1,214.95


In [5]:
n_customers = 1000
inventory_space = 1000
selling = np.array([45.95,
89.9,
99,
47.9,
])

buying = selling - np.random.uniform(10,20)
probabilities = np.exp(-selling / 100) / np.sum(np.exp(-selling/100))
I = range(len(buying))

In [6]:
probabilities

array([0.31120092, 0.20052496, 0.18308284, 0.30519128])

In [7]:
m = gp.Model()
stock = m.addVars(len(I), vtype = GRB.INTEGER)
is_chosen = m.addVars(len(I), vtype = GRB.BINARY)
sold = m.addVars(len(I), vtype = GRB.INTEGER)

m.setObjective(
    gp.quicksum(sold[i]* selling[i] - stock[i]*buying[i] for i in I),
    GRB.MAXIMIZE
)
m.addConstr(gp.quicksum(stock[i] for i in range(4)) <= inventory_space)
m.addConstrs(stock[i] <= is_chosen[i] * n_customers for i in I)
m.addConstrs(sold[i] <= stock[i] for i in I)
m.addConstrs(sold[i] <= n_customers * probabilities[i]  * is_chosen[i] for i in I)

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-26
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i5-12600K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 12 columns and 28 nonzeros
Model fingerprint: 0xfeb29114
Variable types: 0 continuous, 12 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 10958.4 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.095837423707e+04, best bo

In [8]:
stock

{0: <gurobi.Var C0 (value 311.0)>,
 1: <gurobi.Var C1 (value 200.0)>,
 2: <gurobi.Var C2 (value 183.0)>,
 3: <gurobi.Var C3 (value 305.0)>}

In [9]:
buying

array([34.98065642, 78.93065642, 88.03065642, 36.93065642])

In [10]:
selling

array([45.95, 89.9 , 99.  , 47.9 ])

In [11]:
n_customers = 1000
inventory_space = 1000
selling = np.array([45.95,
89.9,
99,
47.9,
])


rating = np.array([3.8, 4.5, 4.8,4.0])
MAX_RATING = 5.

buying = selling - np.random.uniform(10,20)
p1 = np.exp(-selling / 100) / np.sum(np.exp(-selling/100))
p2 = np.exp(-selling*(MAX_RATING - rating) / 100) / np.sum(np.exp(-selling*(MAX_RATING - rating)/100))
I = range(len(buying))

In [12]:
p1

array([0.31120092, 0.20052496, 0.18308284, 0.30519128])

In [13]:
p2

array([0.21709593, 0.24038441, 0.30912302, 0.23339664])

In [14]:
m = gp.Model()
stock = m.addVars(len(I), vtype = GRB.INTEGER)
is_chosen = m.addVars(len(I), vtype = GRB.BINARY)
sold = m.addVars(len(I), vtype = GRB.INTEGER)

m.setObjective(
    gp.quicksum(sold[i]* selling[i] - stock[i]*buying[i] for i in I),
    GRB.MAXIMIZE
)
m.addConstr(gp.quicksum(stock[i] for i in range(4)) <= inventory_space)
m.addConstrs(stock[i] <= is_chosen[i] * n_customers for i in I)
m.addConstrs(sold[i] <= stock[i] for i in I)
m.addConstrs(sold[i] <= n_customers * p2[i]  * is_chosen[i] for i in I)

m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i5-12600K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 12 columns and 28 nonzeros
Model fingerprint: 0x3420432b
Variable types: 0 continuous, 12 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 18693.6 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.869363138130e+04, best bound 1.869363138130e+04, gap 0.0000%


In [15]:
stock

{0: <gurobi.Var C0 (value 217.0)>,
 1: <gurobi.Var C1 (value 240.0)>,
 2: <gurobi.Var C2 (value 309.0)>,
 3: <gurobi.Var C3 (value 233.0)>}

# competitors

In [16]:
SCORE_FACTOR = 1.8
comp = np.array(
    [
    [74,74,74,77.9,89.9],
    [39.24,74,84.9,44.2,163.4]
]
)
r = np.array([
    [3.9,3.9,3.9,3.9,3.9],
    [4.4,4.4,3.3,4.2,4.2]
])

comp_normalised = comp/ max( np.max(comp), np.max(price))
r_normalised = r / MAX_RATING
c_score = r_normalised / comp_normalised
c_score = np.mean(c_score, axis =1)

NameError: name 'price' is not defined

In [ ]:
price = np.array([39.24,74,84.9,44.15,163.4])
price = price / max(price)
ratings = np.array([4,3.9,3.3,4.2,4.4]) / MAX_RATING
score = np.mean(ratings / price)
score

2.062544488727544

In [ ]:
scores = np.asarray([score, 1.64415582, 2.16462918])

In [ ]:
np.exp(scores**SCORE_FACTOR) / np.sum(np.exp(scores**SCORE_FACTOR))

array([0.37196786, 0.10836468, 0.51966746])

------------------------------

# More Data

In [33]:
index = ['product_category_name', 'product_id']
column_min = ['unit_price', 'product_score', 'comp_1', 'ps1', 'comp_2', 'ps2', 'comp_3', 'ps3', 'product_weight_g']
# column_max = []
column_sum = ['customers']

product_category = ['bed_bath_table', 'computers_accessories', 'consoles_games', 'cool_stuff', 'furniture_decor',
                    'garden_tools', 'health_beauty', 'perfumery', 'watches_gifts']

table_min = pd.pivot_table(df, values=column_min, index=index,  aggfunc=np.min)
# table_min = pivot_table(df, values='D', index=index,  aggfunc=np.max)
table_sum = pd.pivot_table(df, values=column_sum, index=index, aggfunc=np.sum)

In [34]:
table = table_min.merge(table_sum, on=index,how='left')
table

comp_1      comp_2      comp_3  \
product_category_name product_id                                       
bed_bath_table        bed1         74.000000   39.240000   39.240000   
                      bed2         74.000000   74.000000   39.240000   
                      bed3         74.000000   84.900000   39.240000   
                      bed4         77.933333   44.154444   39.990000   
                      bed5         89.900000  163.398710   45.950000   
computers_accessories computers1   94.900000   94.900000   77.900000   
                      computers2  114.491154  109.900000   77.900000   
                      computers3  139.990000  134.900000   78.712281   
                      computers4  114.491154  139.990000   77.900000   
                      computers5  114.491154  119.900000   77.900000   
                      computers6  114.491154  149.900000   77.900000   
consoles_games        consoles1    19.900000   19.900000   19.900000   
                      consoles2    19.900000   19.900000   19.900000   
cool_stuff            cool1        85.704286   85.704286   29.500000   
                      cool2        89.863333  116.927500   29.500000   
                      cool3        35.000000   29.500000   29.500000   
                      cool4        89.863333  116.927500   29.500000   
                      cool5        89.863333   99.990000   29.500000   
furniture_decor       furniture1   38.400000   35.000000   38.400000   
                      furniture2   69.900000   35.000000   75.000000   
                      furniture3   38.400000   35.000000   38.400000   
                      furniture4   69.900000   35.000000   78.535714   
garden_tools          garden1      49.900000   49.900000   49.900000   
                      garden10     49.900000   49.900000   49.900000   
                      garden2      49.900000   49.900000   49.900000   
                      garden3      49.900000   49.900000   49.900000   
                      garden4      49.900000   49.900000   49.900000   
                      garden5      49.900000   49.900000   49.900000   
                      garden6      49.900000   49.900000   49.900000   
                      garden7      49.900000   49.900000   49.900000   
                      garden8      49.900000   49.900000   49.900000   
                      garden9      49.900000   49.900000   49.900000   
health_beauty         health1      23.397407   79.990000   58.990000   
                      health10     23.990000   79.990000   58.990000   
                      health2      23.397407   79.990000   58.990000   
                      health3      23.990000   79.990000   58.990000   
                      health4      23.990000   79.990000   58.990000   
                      health5      19.990000   79.990000   58.799524   
                      health6      23.990000   79.990000   58.990000   
                      health7      19.990000   64.990000   58.799524   
                      health8      19.990000   79.990000   58.799524   
                      health9      19.990000   19.990000   58.799524   
perfumery             perfumery1   49.990000   56.990000   49.990000   
                      perfumery2   49.990000  116.906667  116.906667   
watches_gifts         watches1    103.792523  105.000000  132.531250   
                      watches2    103.792523  105.000000  132.531250   
                      watches3     78.000000   78.000000  132.531250   
                      watches4    103.792523  105.000000  132.531250   
                      watches5    103.792523  105.000000  132.531250   
                      watches6    103.792523  105.000000  132.531250   
                      watches7    103.792523  105.000000  132.531250   
                      watches8    103.792523  105.000000  132.531250   

                                  product_score  product_weight_g  ps1  ps2  \
product_category_name product_id                                      

In [19]:
inventory_space = 100000
budget = 1000000
interest_rate = 0.02

In [20]:
# Customers
n_customers = {}
for category in product_category:
    n_customers[category] = (table.loc[category]['customers'].sum())

In [21]:
sum(n_customers.values())

54775

In [22]:
# Selling & Buying & Rating & Probability of buying
unit_price = {}
for category in product_category:
    unit_price[category] = (table.loc[category]['unit_price'].values)

unit_cost = {}
for category in product_category:
    unit_cost[category] = unit_cost[category] * np.random.uniform(0.6,0.7)


rating = {}
for category in product_category:
    rating[category] = (table.loc[category]['product_score'].values)
MAX_RATING = 5.


p2={}
for category in product_category:
    p2[category] = (np.exp(-selling[category]*(MAX_RATING - rating[category]) / 100) 
              / np.sum(np.exp(-selling[category]*(MAX_RATING - rating[category])/100)))


In [23]:
# Store "Attractiveness" Score

import copy

SCORE_FACTOR = 1.8

number_of_comp = 3

comp = {}
comp['price'] = {}
comp['rating'] = {}
comp['score'] = {}

    
for category in product_category:
    comp_price = []
    comp_rating = []
    for i in range(1, 1 + number_of_comp):
        comp_price.append(table.loc[category][f'comp_{i}'].values)
        comp_rating.append(table.loc[category][f'ps{i}'].values)
    comp['price'][category] = np.asarray(comp_price)
    comp['rating'][category] = np.asarray(comp_rating)

    
    
# Normalise data for calculating score
comp_normalised = copy.deepcopy(comp)

for category in product_category:
    comp_normalised['price'][category] = comp_normalised['price'][category] / max(np.max(comp['price'][category]), 
                                                                                  np.max(selling[category]) )
    comp_normalised['rating'][category] = comp_normalised['rating'][category] / MAX_RATING
    comp['score'][category] = comp['rating'][category]/ comp['price'][category]

# for i in range(1, 1 + number_of_comp):
#     for category in product_category:
# comp = np.array(
#     [
#     [74,74,74,77.9,89.9],
#     [39.24,74,84.9,44.2,163.4]
# ]
# )
# r = np.array([
#     [3.9,3.9,3.9,3.9,3.9],
#     [4.4,4.4,3.3,4.2,4.2]
# ])

# comp_normalised = comp/ max( np.max(comp), np.max(price))
# r_normalised = r / MAX_RATING
# c_score = r_normalised / comp_normalised
# c_score = np.mean(c_score, axis =1)


In [24]:
comp

{'price': {'bed_bath_table': array([[ 74.        ,  74.        ,  74.        ,  77.93333333,
           89.9       ],
         [ 39.24      ,  74.        ,  84.9       ,  44.15444444,
          163.3987097 ],
         [ 39.24      ,  39.24      ,  39.24      ,  39.99      ,
           45.95      ]]),
  'computers_accessories': array([[ 94.9      , 114.4911538, 139.99     , 114.4911538, 114.4911538,
          114.4911538],
         [ 94.9      , 109.9      , 134.9      , 139.99     , 119.9      ,
          149.9      ],
         [ 77.9      ,  77.9      ,  78.7122807,  77.9      ,  77.9      ,
           77.9      ]]),
  'consoles_games': array([[19.9, 19.9],
         [19.9, 19.9],
         [19.9, 19.9]]),
  'cool_stuff': array([[ 85.70428571,  89.86333333,  35.        ,  89.86333333,
           89.86333333],
         [ 85.70428571, 116.9275    ,  29.5       , 116.9275    ,
           99.99      ],
         [ 29.5       ,  29.5       ,  29.5       ,  29.5       ,
           29.5       ]

In [25]:
comp_normalised['price'][1]['bed_bath_table']

KeyError: 1

In [33]:
m = gp.Model()

BIGNUM = 1e6 # sum(n_customers.values())

T = range(4)

stock = {}
is_chosen = {}
sold = {}
discount = {}
free_shipping = {}

SHIPPING_COST = 5.
MARKET_CUSTOMERS = {category: 10000 for category in product_category}
DISCOUNT_FACTOR = 0.02
FREE_SHIPPING_FACTOR = 0.05

for category in product_category:
    I = range(len(selling[category]))
    stock[category] = m.addVars(I, T, vtype = GRB.INTEGER)
    is_chosen[category] = m.addVars(I, T, vtype = GRB.BINARY)
gp.quicksum(stock['bed_bath_table'])

<gurobi.LinExpr: <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>>

In [32]:
m = gp.Model()

BIGNUM = 1e6 # sum(n_customers.values())

T = range(4)

stock = {}
is_chosen = {}
sold = {}
discount = {}
free_shipping = {}

SHIPPING_COST = 5
MARKET_CUSTOMERS = {category: 10000 for category in product_category}
DISCOUNT_FACTOR = 5
FREE_SHIPPING_FACTOR = 0.1
BUDGET = 1e9

for category in product_category:
    I = range(len(selling[category]))
    stock[category] = m.addVars(I,vtype = GRB.INTEGER)
    is_chosen[category] = m.addVars(I, vtype = GRB.BINARY)
    sold[category] = m.addVars(I,vtype = GRB.INTEGER)
    discount[category] = m.addVars(I,vtype=GRB.CONTINUOUS, lb=0, ub=1)
    free_shipping[category] = m.addVars(I,  vtype = GRB.BINARY)

nomralised_prices = {
    k : arr / np.max(arr) for k, arr in selling.items()
}

captured_customers = lambda category,i: (
    MARKET_CUSTOMERS[category] * (FREE_SHIPPING_FACTOR * free_shipping[category][i]
                                  + DISCOUNT_FACTOR * discount[category][i])
)
amount_sold  = lambda category,i: p2[category][i]* (n_customers[category] + captured_customers(category,i))

m.setObjective(
    gp.quicksum(gp.quicksum(sold[category][i] * selling[category][i] * (1 - discount[category][i]) 
                            - stock[category][i] * buying[category][i] * (1 + interest_rate)
                            - sold[category][i] * SHIPPING_COST * free_shipping[category][i]
                for i in range(len(selling[category]))) for category in product_category),
    GRB.MAXIMIZE
)

# Can't store past inventory space
m.addConstrs( gp.quicksum(stock[category][i] for i in range(len(selling[category]))) 
                         <= inventory_space for category in product_category)

# Can't buy more than the budget
m.addConstr(gp.quicksum( gp.quicksum(stock[category][i] * buying[category][i] for i in range(len(selling[category]))) 
                        for category in product_category) <= BUDGET)

for category in product_category:
    # If a product is stocked, is_chosen is set to be 1 [Change to BIGNUM to remove dependencies on n_customers]
    m.addConstrs(stock[category][i] <= is_chosen[category][i] * BIGNUM
                 for i in range(len(selling[category])))
    # Can't sell more than you have stocked
    m.addConstrs(sold[category][i] <= stock[category][i] for i in range(len(selling[category])))
    # How much we can sell depends on number of customers [Removed is_chosen]
    m.addConstrs(sold[category][i] <= amount_sold(category, i)
                 for i in range(len(selling[category])))
    m.addConstrs((FREE_SHIPPING_FACTOR * free_shipping[category][i]
                                  + DISCOUNT_FACTOR * discount[category][i]) <= 1 for i in range(len(selling[category])))

m.params.NonConvex = 2
m.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 218 rows, 260 columns and 572 nonzeros
Model fingerprint: 0xbc23d2f1
Model has 104 quadratic objective terms
Variable types: 52 continuous, 208 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  Objective range  [1e+01, 3e+02]
  QObjective range [1e+01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
Found heuristic solution: objective -0.0000000
Presolve removed 216 rows and 257 columns
Presolve time: 1.31s
Presolved: 6 rows, 6 columns, 15 nonzeros
Presolved model has 1 bilinear constraint(s)
Found heuristic solution: objective 1953693.9838
Variable types: 3 continuous, 3 integer (1 binary)
Found heuristic solution: objective 2010954.6330

Root

In [103]:
discount

{'bed_bath_table': {0: <gurobi.Var C15 (value 0.12812209677587985)>,
  1: <gurobi.Var C16 (value 0.12817323080563459)>,
  2: <gurobi.Var C17 (value 0.1281978336804244)>,
  3: <gurobi.Var C18 (value 0.12813580486383638)>,
  4: <gurobi.Var C19 (value 0.12810443665931506)>},
 'computers_accessories': {0: <gurobi.Var C43 (value 0.145278284252246)>,
  1: <gurobi.Var C44 (value 0.1452470477595548)>,
  2: <gurobi.Var C45 (value 0.14533930135862214)>,
  3: <gurobi.Var C46 (value 0.14534306645146378)>,
  4: <gurobi.Var C47 (value 0.14526741642517965)>,
  5: <gurobi.Var C48 (value 0.14532024297363247)>},
 'consoles_games': {0: <gurobi.Var C61 (value 0.16732483151113992)>,
  1: <gurobi.Var C62 (value 0.16735610162461545)>},
 'cool_stuff': {0: <gurobi.Var C80 (value 0.12439801677913158)>,
  1: <gurobi.Var C81 (value 0.12445742087135224)>,
  2: <gurobi.Var C82 (value 0.12446364943482838)>,
  3: <gurobi.Var C83 (value 0.1243142202795574)>,
  4: <gurobi.Var C84 (value 0.12447355442615748)>},
 'furnit

In [104]:
free_shipping

{'bed_bath_table': {0: <gurobi.Var C20 (value 0.0)>,
  1: <gurobi.Var C21 (value -0.0)>,
  2: <gurobi.Var C22 (value -0.0)>,
  3: <gurobi.Var C23 (value 0.0)>,
  4: <gurobi.Var C24 (value 0.0)>},
 'computers_accessories': {0: <gurobi.Var C49 (value 0.0)>,
  1: <gurobi.Var C50 (value 0.0)>,
  2: <gurobi.Var C51 (value -0.0)>,
  3: <gurobi.Var C52 (value -0.0)>,
  4: <gurobi.Var C53 (value 0.0)>,
  5: <gurobi.Var C54 (value 0.0)>},
 'consoles_games': {0: <gurobi.Var C63 (value 0.0)>,
  1: <gurobi.Var C64 (value 0.0)>},
 'cool_stuff': {0: <gurobi.Var C85 (value 0.0)>,
  1: <gurobi.Var C86 (value 0.0)>,
  2: <gurobi.Var C87 (value 0.0)>,
  3: <gurobi.Var C88 (value 0.0)>,
  4: <gurobi.Var C89 (value 0.0)>},
 'furniture_decor': {0: <gurobi.Var C106 (value 0.0)>,
  1: <gurobi.Var C107 (value -0.0)>,
  2: <gurobi.Var C108 (value 0.0)>,
  3: <gurobi.Var C109 (value -0.0)>},
 'garden_tools': {0: <gurobi.Var C150 (value 0.0)>,
  1: <gurobi.Var C151 (value 0.0)>,
  2: <gurobi.Var C152 (value 0.0)

In [89]:
stock

{'bed_bath_table': {0: <gurobi.Var C0 (value 9416.0)>,
  1: <gurobi.Var C1 (value 6177.0)>,
  2: <gurobi.Var C2 (value 3292.0)>,
  3: <gurobi.Var C3 (value 9792.0)>,
  4: <gurobi.Var C4 (value 5230.0)>},
 'computers_accessories': {0: <gurobi.Var C25 (value 9463.0)>,
  1: <gurobi.Var C26 (value 5711.0)>,
  2: <gurobi.Var C27 (value 6341.0)>,
  3: <gurobi.Var C28 (value 7352.0)>,
  4: <gurobi.Var C29 (value 5775.0)>,
  5: <gurobi.Var C30 (value 6434.0)>},
 'consoles_games': {0: <gurobi.Var C55 (value 17798.0)>,
  1: <gurobi.Var C56 (value 16767.0)>},
 'cool_stuff': {0: <gurobi.Var C65 (value 6027.0)>,
  1: <gurobi.Var C66 (value 5277.0)>,
  2: <gurobi.Var C67 (value 10023.0)>,
  3: <gurobi.Var C68 (value 2368.0)>,
  4: <gurobi.Var C69 (value 6613.0)>},
 'furniture_decor': {0: <gurobi.Var C90 (value 9010.0)>,
  1: <gurobi.Var C91 (value 7570.0)>,
  2: <gurobi.Var C92 (value 9378.0)>,
  3: <gurobi.Var C93 (value 8865.0)>},
 'garden_tools': {0: <gurobi.Var C110 (value 4198.0)>,
  1: <gurobi

In [75]:
selling['health_beauty'] - buying['health_beauty']

array([ 32.50069338,  18.10994724, 124.28197846,  42.44325166,
        11.43394202, 123.18690647,  23.70535337,  22.48529585,
        30.58866294,   7.64429769])